In [1]:
# THE COMPILER : LEXER & PARSER

In [2]:
# What kind of tokens does Python have?
import token

token.tok_name

{0: 'ENDMARKER',
 1: 'NAME',
 2: 'NUMBER',
 3: 'STRING',
 4: 'NEWLINE',
 5: 'INDENT',
 6: 'DEDENT',
 7: 'LPAR',
 8: 'RPAR',
 9: 'LSQB',
 10: 'RSQB',
 11: 'COLON',
 12: 'COMMA',
 13: 'SEMI',
 14: 'PLUS',
 15: 'MINUS',
 16: 'STAR',
 17: 'SLASH',
 18: 'VBAR',
 19: 'AMPER',
 20: 'LESS',
 21: 'GREATER',
 22: 'EQUAL',
 23: 'DOT',
 24: 'PERCENT',
 25: 'LBRACE',
 26: 'RBRACE',
 27: 'EQEQUAL',
 28: 'NOTEQUAL',
 29: 'LESSEQUAL',
 30: 'GREATEREQUAL',
 31: 'TILDE',
 32: 'CIRCUMFLEX',
 33: 'LEFTSHIFT',
 34: 'RIGHTSHIFT',
 35: 'DOUBLESTAR',
 36: 'PLUSEQUAL',
 37: 'MINEQUAL',
 38: 'STAREQUAL',
 39: 'SLASHEQUAL',
 40: 'PERCENTEQUAL',
 41: 'AMPEREQUAL',
 42: 'VBAREQUAL',
 43: 'CIRCUMFLEXEQUAL',
 44: 'LEFTSHIFTEQUAL',
 45: 'RIGHTSHIFTEQUAL',
 46: 'DOUBLESTAREQUAL',
 47: 'DOUBLESLASH',
 48: 'DOUBLESLASHEQUAL',
 49: 'AT',
 50: 'ATEQUAL',
 51: 'RARROW',
 52: 'ELLIPSIS',
 53: 'OP',
 54: 'AWAIT',
 55: 'ASYNC',
 56: 'ERRORTOKEN',
 57: 'COMMENT',
 58: 'NL',
 59: 'BACKQUOTE',
 60: 'COMMENT',
 256: 'NT_OFFSET'}

In [3]:
# How are tokens used?
s = open('test.py').read()
s
# test.py에 내가 작성한 것은 하나의 문자열일 뿐이다.

'def func(x, y):\n\treturn x + y\n\na = 10\nb = 20\n\nc = func(a, b)\nprint(c)'

In [4]:
from tokenize import tokenize
from io import BytesIO

In [5]:
g = tokenize(BytesIO(s.encode('utf-8')).readline)
g

<generator object _tokenize at 0x00000198F96206D0>

In [6]:
# See lexer turn test.py into tokens
for token in g:
    print(token)

TokenInfo(type=59 (BACKQUOTE), string='utf-8', start=(0, 0), end=(0, 0), line='')
TokenInfo(type=1 (NAME), string='def', start=(1, 0), end=(1, 3), line='def func(x, y):\n')
TokenInfo(type=1 (NAME), string='func', start=(1, 4), end=(1, 8), line='def func(x, y):\n')
TokenInfo(type=53 (OP), string='(', start=(1, 8), end=(1, 9), line='def func(x, y):\n')
TokenInfo(type=1 (NAME), string='x', start=(1, 9), end=(1, 10), line='def func(x, y):\n')
TokenInfo(type=53 (OP), string=',', start=(1, 10), end=(1, 11), line='def func(x, y):\n')
TokenInfo(type=1 (NAME), string='y', start=(1, 12), end=(1, 13), line='def func(x, y):\n')
TokenInfo(type=53 (OP), string=')', start=(1, 13), end=(1, 14), line='def func(x, y):\n')
TokenInfo(type=53 (OP), string=':', start=(1, 14), end=(1, 15), line='def func(x, y):\n')
TokenInfo(type=4 (NEWLINE), string='\n', start=(1, 15), end=(1, 16), line='def func(x, y):\n')
TokenInfo(type=5 (INDENT), string='\t', start=(2, 0), end=(2, 1), line='\treturn x + y\n')
TokenInfo(

In [7]:
# See how parser turns the tokens into the AST
import ast

In [8]:
node = ast.parse(s)
node

In [9]:
g = ast.walk(node)
# walk 함수는 각 node를 root부터 한 단계식 내려가며 (code를 위에서부터 아래로 내려가며) 해당 단계의 node를 왼쪽에서 오른쪽으로 이동하며 보여주고, 다음 단계로 내려간다.

for node in g:
    print(node)

    # Module이 root, AST의 근원
    # FunctionDef는 2단계의 첫번째, 함수를 정의하는 "def func"

In [10]:
# How is the symbol table made from the AST?
    # The symbol table is a table with information about all the variables and functions in a source code
        # The global symbol table holds information about variables and functions in the global area.
        # The function symbol table holds similar information in a function's area
    # The symbol table's existence explains how the stack frame is able to create in advance slots for the variables in a function,
        # even though values for those variables have not been inputted

In [11]:
import symtable

In [12]:
s = open('test.py').read()
sym = symtable.symtable(s, 'test.py', 'exec')
    # 'exec' means expressions & statements can be used,
        # 'eval' can be used instead but can only be used with expressions
sym
# top은 global영역의 symbol table을 의미한다

<SymbolTable for top in test.py>

In [13]:
# Checking the function symbol table
sym.get_name()
# If 'top' is the output, this means the "sym", current symbol table, is the global symbol table

'top'

In [14]:
sym.get_symbols()
# Allows the user to check what symbols are in "sym", the global symbol table
    # The symbols 'a' & 'b' refer to global variables a & b

[<symbol 'func'>, <symbol 'a'>, <symbol 'b'>, <symbol 'c'>, <symbol 'print'>]

In [15]:
sym.has_children()
# Checks whether any of the symbols in "sym" has its own subordinated symbols (a.k.a. children)

True

In [16]:
sym.get_children()
# Checks which symbol(s) in "sym" has its own symbols

[<Function SymbolTable for func in test.py>]

In [17]:
func_sym = sym.get_children()[0]
print(func_sym.get_name())
    # Check the name of the symbol(s) in "sym" that has its own symbols
print(func_sym.get_symbols())
    # Check what symbols this symbol with children has
    
    # BTW print is a function and thus would have children

func
[<symbol 'x'>, <symbol 'y'>]


In [18]:
# How is the bytecode made from the AST?

In [19]:
import dis

In [20]:
# THIS IS BYTECODE

dis.dis(s)

# LOAD_CONST, MAKE_FUNCTION, STORE_NAME 등등은 instruction의 명령어와 동일하다... 다만 기계어가 아닌 형태일뿐
    
    #     4           8 LOAD_CONST               2 (10)
        # "4"은 해당 명령어의 값이 실제파일에서 몇 번째 줄에 위치하는지 알려준다
        # "8"는 실제파일을 compile해서 나온 bytecode에서, 해당 명령어가 앞에서 몇 번째인지 offset을 알려준다
        # LOAD_CONST는 "co_consts[consti]를 stack으로 내보내라"라는 의미의 명령어이다
        # () 앞에 있는 "2"는 code.co_consts에서 () 값의 index번호를 알려준다
        # () 속 내용은 실제파일에서 명령어의 값이 무엇인지 알려준다
        
    # BTW it's ok to not understand all of this, because this is about the inner workings of Python that go on below the surface...
        # It's pretty deep and complex material.

  1           0 LOAD_CONST               0 (<code object func at 0x00000198F9622AE0, file "<dis>", line 1>)
              2 LOAD_CONST               1 ('func')
              4 MAKE_FUNCTION            0
              6 STORE_NAME               0 (func)

  4           8 LOAD_CONST               2 (10)
             10 STORE_NAME               1 (a)

  5          12 LOAD_CONST               3 (20)
             14 STORE_NAME               2 (b)

  7          16 LOAD_NAME                0 (func)
             18 LOAD_NAME                1 (a)
             20 LOAD_NAME                2 (b)
             22 CALL_FUNCTION            2
             24 STORE_NAME               3 (c)

  8          26 LOAD_NAME                4 (print)
             28 LOAD_NAME                3 (c)
             30 CALL_FUNCTION            1
             32 POP_TOP
             34 LOAD_CONST               4 (None)
             36 RETURN_VALUE


In [21]:
g = dis.get_instructions(s)

In [22]:
for inst in g:
    print(inst.opname.ljust(20), end = '  ')
        # opname, 즉 instruction을 구성하는 각 명령어를 출력하라는 말
    print(inst.argval)
        # 명령어의 값, 즉 "무엇을 하라는" 명령어인지를 출력하라는 말

LOAD_CONST            <code object func at 0x00000198F9622AE0, file "<disassembly>", line 1>
LOAD_CONST            func
MAKE_FUNCTION         0
STORE_NAME            func
LOAD_CONST            10
STORE_NAME            a
LOAD_CONST            20
STORE_NAME            b
LOAD_NAME             func
LOAD_NAME             a
LOAD_NAME             b
CALL_FUNCTION         2
STORE_NAME            c
LOAD_NAME             print
LOAD_NAME             c
CALL_FUNCTION         1
POP_TOP               None
LOAD_CONST            None
RETURN_VALUE          None


In [23]:
# 명령어를 Code Object로 변환한다
code = compile(s, 'test.py', 'exec')
code

<code object <module> at 0x00000198F965DC90, file "test.py", line 1>

In [24]:
code.co_code
# 'test.py' 내용을 16진법 코드로 변환한 것

b'd\x00d\x01\x84\x00Z\x00d\x02Z\x01d\x03Z\x02e\x00e\x01e\x02\x83\x02Z\x03e\x04e\x03\x83\x01\x01\x00d\x04S\x00'

In [25]:
len(code.co_code)
# code.co_code의 길이는 38byte

38

In [26]:
code.co_consts

(<code object func at 0x00000198F9622AE0, file "test.py", line 1>,
 'func',
 10,
 20,
 None)

In [27]:
# How is the bytecode turned into machine code (so it can be executed by the CPU)?
    # Instructions like LOAD_CONST are the bytecode for Python, not machine code
    # The answer is the Python Virtual Machine (PVM)
        # The PVM is basically an infinite for loop, like "while True:"
        # If the bytecode is LOAD_CONST, this loop executes C code "A"... if the bytecode is LOAD_FAST, this loop executes C code "B"... etc.

In [28]:
# DATA STRUCTURES & ALGORITHMS

In [29]:
# Linear Search & Binary Search, and the Big O

# Objective: Compare performance of Linear Search vs. Binary Search
    # Linear Search: Basically a for loop

In [30]:
# Linear Search

# Write "linear_search(data, target)" & you get the index of the target
def linear_search(data, target):
    for e in data:
        if e == target:
            return data.index(e)
    return None

In [31]:
li = [4, 2, 7, 8, 9, 10]
idx = linear_search(li, 10)
    # 10을 찾기 위해 for loop을 6번 순회해야한다
print(idx)
if idx:
    print(li[idx])
else:
    print("No such data")

5
10


In [32]:
# Binary Search
    # To use binary search, the list must always be ordered first
    # When using binary search, you first go to the middle of the list & consider whether your target is bigger or smaller than the middle.
        # If the target is smaller, the middle & everything bigger than the middle is unneeded and  not considered.
            # Rinse & repeat until you find your target

In [33]:
# Write "binary_search(data, target)" & you get the index of the target
    # Return "None" if target is not in data

# My attempt to write this program
def binary_search(data, target):
    sta = data[0]
    end = data[-1]
    mid = end//2
    while True:
        if mid == target:
            return data.index(e)
        elif mid < target:
            sta = data[mid+1]
        elif mid > target:
            end = data[mid-1]

In [34]:
# 강사의 모범답안
def binary_search(data, target):
    data.sort()
        # 혹시 사용자가 정렬되지 않은 data를 입력할 것에 대비하여
    start = 0
    end = len(data) - 1
    
    while start <= end:
        mid = (start + end)//2
        if target == data[mid]:
            return mid
        elif target < data[mid]:
            end = mid - 1
        else:
            start = mid + 1
    return None

In [35]:
li = [i**2 for i in range(10)]
li

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [36]:
target = 82
idx = binary_search(li, target)
if idx:
    print(li[idx])
else:
    print("No data")

No data


In [37]:
# Recursion (재귀함수)
    # 1. 함수 정의 안에 같은 함수로 호출해야
    # 2. 탈출 조건이있어야

In [38]:
# Recursion with Factorial
    # Factorial (계승)
    
    # ex) 3! = 3 * 2 * 1
        # 4! = 4 * 3 * 2 * 1
        # 5! = 5 * 4 * 3 * 2 * 1

def factorial(n):
    return n * factorial(n-1)
        # 탈출조건이 없다!!!

factorial(5)
# 에러명 "maximum recursion depth exceeded" == "stack overflow"
    # 실행을 하면 factorial(n) stack frame 생성 => 그 위에 factorial(n-1) stack frame 생성
    # => 그 위에 facctorial(n-1-1) stack frame 생성 => 무한반복!!!
        # 하지만 stack은 static memory allocation(SMA)이기에 메모리 용량이 정해져있다... 이것을 초과하면 overflow한다

RecursionError: maximum recursion depth exceeded

In [40]:
# Recursion with Factorial with 탈출조건

def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n-1)


factorial(3)

6

In [41]:
# Recursion with Fibonnaci
    # n = 0 or n = 1이면, 값이 1
    # v(n) = v(n-2) + v(n-1)
        
def fibonacci(n):
    if n == 0 or n == 1:
        return 1
    return fibonacci(n-2) + fibonacci(n-1)

In [42]:
for i in range(10):
    print(fibonacci(i), end = '  ')

1  1  2  3  5  8  13  21  34  55  

In [43]:
# Recursion with Hanoi Tower

# n : 쟁반 개수
# from, by, to : A, B, C

# Apparently this should be enough hints to solve this thing.....

def hanoi(from_, by_, to_, n):
    # 탈출조건
    if n == 1:
        pass

    print('{}에서 {}로 {}번째 쟁반 이동'.format(from_, to_, n))

In [44]:
# 강사의 모범답안

# This is the hanoi code
def hanoi(from_, by_, to_, n):
    # 탈출조건
    if n == 1:
        print('{}에서 {}로 {}번째 쟁반 이동'.format(from_, to_, n))
        return
    hanoi(from_, to_, by_, n-1)
    print('{}에서 {}로 {}번째 쟁반 이동'.format(from_, to_, n))
    hanoi(by_, from_, to_, n-1)

# This is the test code to test whether the hanoi code works properly
while 1:
    numOfTray = int(input("원반의 개수를 입력하세요!(종료 : 0) :"))
    if numOfTray == 0:
        break
    hanoi('A', 'B', 'C', numOfTray)

원반의 개수를 입력하세요!(종료 : 0) :3
A에서 C로 1번째 쟁반 이동
A에서 B로 2번째 쟁반 이동
C에서 B로 1번째 쟁반 이동
A에서 C로 3번째 쟁반 이동
B에서 A로 1번째 쟁반 이동
B에서 C로 2번째 쟁반 이동
A에서 C로 1번째 쟁반 이동
원반의 개수를 입력하세요!(종료 : 0) :0


In [45]:
# Do a binary search with recursion

# The original binary search function using while
def binary_search(data, target):
    data.sort()
    # 첫 번째 인덱스
    start = 0
    # 마지막 인덱스
    end = len(data) - 1
    
    while start <= end:
        mid = (start + end) // 2
        if target == data[mid]:
            return mid
        elif target < data[mid]:
            end = mid - 1
        else:
            start = mid + 1
    return None

In [46]:
# The interface for doing a binary search with recursion
def binary_search(data, target, start, end):
    # 1. 탈출조건
    if ....

    ......
    
    return binary_search(...)

SyntaxError: invalid syntax (<ipython-input-46-97538888cad8>, line 4)

In [47]:
# My attempt to do a binary search with recursion

def binary_search(data, target, start, end):
    # 1. 탈출조건
    if start >= end:
        return None

    mid = (start + end) // 2
    if target == data[mid]:
        return mid
    
    binary_search(data, target, mid + 1, end)
    print
    binary_search(data, target, start, mid -1)
    
    return binary_search(...)

In [48]:
# 강사의 모범답안

def binary_search(data, target, start, end):
    # 1. 탈출조건
    if start > end:
        return None
    
    mid = (start + end) // 2
    
    if target == data[mid]:
        return mid
    elif target < data[mid]:
        end = mid - 1
    else:
        start = mid + 1

    return binary_search(data, target, start, end)

In [50]:
data = [i**2 for i in range(1, 11)]
        
target = 4
start = 0
end = len(data) -1
    
idx = binary_search(data, target, start, end)

if idx == None:
    print("{}이 존재하지 않습니다".format(target))
else:
    print("찾는 데이터의 인덱스는 {}이고 데이터는 {} 입니다".format(idx, data[idx]))

찾는 데이터의 인덱스는 1이고 데이터는 4 입니다


In [56]:
# Exercise)) n을 입력하면, 1 + 2 + 3 + 4 + ... ...+ n을 연산하게 하는 재귀함수를 설계하라.
    # 함수 sum_to_n(n) -> interger(sum이 나오도록)

# My attempt to do this exercise
total = 0

def sum_to_n(n):
    if n == 0:
        print(total)
        return
    # 탈출조건
    # 함수 내부에서 sum_to_n()
    total += n
    sum_to_n(n-1)

In [57]:
print(sum_to_n(10))

UnboundLocalError: local variable 'total' referenced before assignment

In [58]:
# 강사의 모범답안
def sum_to_n(n):
    if n == 1:
        return 1

    return sum_to_n(n-1) + n

In [59]:
print(sum_to_n(10))

55


In [61]:
# 누군가의 면접에서의 질문

# 실제로 면접관이 물어보는 것:
    # 자료구조를 아는가?
    # recursion을 아는가?
    # 개발자에게 필요한 수학을 할 줄 아는가?
# 해법: 등차수열 --> 버블소트의 빅오

In [65]:
# 일반적으로 이렇게 푼다... 하지만 면접에서는 반복문을 사용하지 말라고 했다...
res = 0
for i in range(1, 100000001):
    res += 1
print(sum_to_num(100000000))

5000000050000000


In [64]:
# 등차수열을 이용한 해법
def sum_to_num(n):
    return (n * (1 + n)) // 2
print(sum_to_num(100000000))

5000000050000000
